In [138]:
import requests
import json
import pandas as pd
import numpy as np
import os
import dotenv as load_dotenv
import time 
import pycoingecko


load_dotenv.load_dotenv()

api_key = os.getenv("COINGECKO_API")

print(api_key)

CG-nm5ynnbGBWTugQhVY2vNMWbP


In [139]:
pd.set_option('display.max_colwidth', None)  
pd.set_option('display.max_columns', None)   
pd.set_option('display.width', 200)    
pd.set_option('display.float_format', '{:.8f}'.format)

# On-Chain DEX Endpoints (Coingecko)

In [140]:

# url = "https://pro-api.coingecko.com/api/v3/onchain/networks/arbitrum-one/pools"

url = "https://pro-api.coingecko.com/api/v3/onchain/networks"

headers = {
           "accept": "application/json",
           "x-cg-pro-api-key": api_key
           }

response = requests.get(url, headers=headers)

data = response.json()

networks = [
    {
        "id": item["id"],
        "name": item["attributes"]["name"],
        "asset_platform_id": item["attributes"]["coingecko_asset_platform_id"]
    }
    for item in data["data"]
]

df = pd.DataFrame(networks)
print(df)

# print(response.status_code)
# print(response.text)

               id           name    asset_platform_id
0             eth       Ethereum             ethereum
1             bsc      BNB Chain  binance-smart-chain
2     polygon_pos    Polygon POS          polygon-pos
3            avax      Avalanche            avalanche
4            movr      Moonriver            moonriver
..            ...            ...                  ...
95       humanode       Humanode             humanode
96      mxc-zkevm      MXC zkEVM            moonchain
97         zkfair         ZKFair               zkfair
98     alveychain     AlveyChain           alveychain
99  hypra-network  Hypra Network        hypra-network

[100 rows x 3 columns]


In [141]:

url = "https://pro-api.coingecko.com/api/v3/onchain/networks/arbitrum/pools/"

headers = {
           "accept": "application/json",
           "x-cg-pro-api-key": api_key
           }

response = requests.get(url, headers=headers)

data = response.json()



print(response.status_code)
print(response.text)

200
{"data":[{"id":"arbitrum_0x6f38e884725a116c9c7fbf208e79fe8828a2595f","type":"pool","attributes":{"base_token_price_usd":"1.0017057214403","base_token_price_native_currency":"0.00040172041754003","quote_token_price_usd":"2493.88","quote_token_price_native_currency":"1.0","base_token_price_quote_token":"0.0004017204175","quote_token_price_base_token":"2489.29343976","address":"0x6f38e884725a116c9c7fbf208e79fe8828a2595f","name":"USDC / WETH 0.01%","pool_created_at":"2023-06-10T16:44:01Z","fdv_usd":"4348792659.75579","market_cap_usd":"60637468660.8295","price_change_percentage":{"m5":"0.26","m15":"0.26","m30":"0.43","h1":"0.21","h6":"0.39","h24":"0.18"},"transactions":{"m5":{"buys":169,"sells":142,"buyers":53,"sellers":39},"m15":{"buys":530,"sells":443,"buyers":103,"sellers":79},"m30":{"buys":1346,"sells":1026,"buyers":174,"sellers":122},"h1":{"buys":2105,"sells":1668,"buyers":225,"sellers":186},"h6":{"buys":12166,"sells":10407,"buyers":659,"sellers":647},"h24":{"buys":43698,"sells":37

In [142]:
pool_data = data["data"]
flattened = []

for pool in pool_data:
    entry = pool["attributes"].copy()
    entry["id"] = pool["id"]
    entry["base_token_id"] = pool["relationships"]["base_token"]["data"]["id"]
    entry["quote_token_id"] = pool["relationships"]["quote_token"]["data"]["id"]
    entry["dex_id"] = pool["relationships"]["dex"]["data"]["id"]
    flattened.append(entry)


df = pd.DataFrame(flattened)


print(df.head())
#print(df)

  base_token_price_usd base_token_price_native_currency quote_token_price_usd quote_token_price_native_currency base_token_price_quote_token quote_token_price_base_token  \
0      1.0017057214403              0.00040172041754003               2493.88                               1.0              0.0004017204175                2489.29343976   
1     1.00151473205561              0.00040172041754003               2493.88                               1.0              0.0004017204175                2489.29343976   
2      1.0017428747737              0.00040172041754003               2493.88                               1.0              0.0004017204175                2489.29343976   
3     1.00195097940747              0.00040168195327911               2494.15                               1.0              0.0004016819533               2489.531809524   
4              2493.88                              1.0      1.00210627738907              0.000401878994743886               2488.3111

In [143]:
df.columns.to_list()

['base_token_price_usd',
 'base_token_price_native_currency',
 'quote_token_price_usd',
 'quote_token_price_native_currency',
 'base_token_price_quote_token',
 'quote_token_price_base_token',
 'address',
 'name',
 'pool_created_at',
 'fdv_usd',
 'market_cap_usd',
 'price_change_percentage',
 'transactions',
 'volume_usd',
 'reserve_in_usd',
 'id',
 'base_token_id',
 'quote_token_id',
 'dex_id']

In [144]:
# Filter the DataFrame where dex_id is 'uniswap_v3_arbitrum'
uniswap_df = df[df['dex_id'] == 'uniswap_v3_arbitrum']

# Optionally, display or inspect the result
print(uniswap_df)


   base_token_price_usd base_token_price_native_currency quote_token_price_usd quote_token_price_native_currency base_token_price_quote_token quote_token_price_base_token  \
0       1.0017057214403              0.00040172041754003               2493.88                               1.0              0.0004017204175                2489.29343976   
3      1.00195097940747              0.00040168195327911               2494.15                               1.0              0.0004016819533               2489.531809524   
5               2493.88                              1.0      1.00126713397985              0.000401878994743886               2488.311190878              0.0004018789947   
6      105853.324056788                  42.440552772335               2494.15                               1.0                42.4405527723                0.02356236983   
8      1.00184251489473              0.00040172041754003      105841.865083793                  42.4406407220045            0.0000

In [145]:
# List of target pool IDs
target_pools = [
    "arbitrum_0x377bece457fb0bb318c8c6a86c4833a8af2dfad1",
    "arbitrum_0x0b5a972b27c5b735990c28011ccb4fd93966120c",
    "arbitrum_0xf89239aee0335f302c310b3bbad280be479be17f"
]

# Filter the DataFrame
filtered_df = df[df['id'].isin(target_pools)]

filtered_df.head()

,base_token_price_usd,base_token_price_native_currency,quote_token_price_usd,quote_token_price_native_currency,base_token_price_quote_token,quote_token_price_base_token,address,name,pool_created_at,fdv_usd,market_cap_usd,price_change_percentage,transactions,volume_usd,reserve_in_usd,id,base_token_id,quote_token_id,dex_id


## Get Token Metadata/pools

In [146]:
url = "https://pro-api.coingecko.com/api/v3/onchain/networks/arbitrum/tokens/0x050C24dBf1eEc17babE5fc585F06116A259CC77A"


headers = {
           "accept": "application/json",
           "x-cg-pro-api-key": api_key
           }

response = requests.get(url, headers=headers)

data = response.json()


# Extract attributes and pools
attributes = data["data"]["attributes"]
pools = data["data"]["relationships"]["top_pools"]["data"]

# Convert attributes to DataFrame (1 row)
attributes_df = pd.DataFrame([attributes])


pools_df = pd.DataFrame(pools)


pools_df["token_name"] = attributes["name"]
pools_df["token_symbol"] = attributes["symbol"]


print("Attributes DataFrame:\n", attributes_df)
print("\nTop Pools DataFrame:\n", pools_df)





# print(response.status_code)
# print(response.text)

Attributes DataFrame:
                                       address    name  symbol  decimals                                                                            image_url coingecko_coin_id  total_supply  \
0  0x050c24dbf1eec17babe5fc585f06116a259cc77a  dlcBTC  DLCBTC         8  https://coin-images.coingecko.com/coins/images/37289/large/iBTC_main.png?1742022373   dlc-link-dlcbtc  2461763181.0   

         price_usd           fdv_usd      total_reserve_in_usd                  volume_usd   market_cap_usd  
0  105249.16834461  2590985.27461632  1405706.1379069748553513  {'h24': '490.95754568622'}  3744455.1530453  

Top Pools DataFrame:
                                                     id  type token_name token_symbol
0  arbitrum_0x377bece457fb0bb318c8c6a86c4833a8af2dfad1  pool     dlcBTC       DLCBTC
1  arbitrum_0x0b5a972b27c5b735990c28011ccb4fd93966120c  pool     dlcBTC       DLCBTC
2  arbitrum_0xf89239aee0335f302c310b3bbad280be479be17f  pool     dlcBTC       DLCBTC
